# モデルを解釈する

Azure Machine Learning を使用して、各機能が予測ラベルに与える影響の量を定量化する *Explainer* を使用して、モデルを解釈できます。一般的な Explainer は多く、それぞれ異なる種類のモデリング アルゴリズムに適しています。ただし、それらを使用する基本的なアプローチは同じです。

## モデルを説明する

Azure Machine Learning の外部でトレーニングされたモデルから始めましょう - 下のセルを実行して、デシジョン ツリー分類モデルをトレーニングします。

In [ ]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 糖尿病データセットを読み込む
print("Loading Data...")
data = pd.read_csv('data/diabetes.csv')

# フィーチャーとラベルを分離する
features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']
X, y = data[features].values, data['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# デシジョン ツリー モデルをトレーニングする
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

print('Model trained.')

トレーニング プロセスでは、ホールドバック検証データセットに基づいてモデル評価メトリクスが生成されるため、予測の精度を把握できます。しかし、データのフィーチャーは予測にどのような影響を与えるのでしょうか?

### Azure ML Interpretability ライブラリ をインストールする
確認するには、まず Azure ML Interpretability ライブラリをインストールします。これを使用すると、Azure ML 実験でトレーニングされていない場合や、Azure ML ワークスペースに登録されていない場合でも、多くの一般的な種類のモデルを解釈できます。

In [ ]:
!pip install --upgrade azureml-interpret

### モデルの説明を取得する

ライブラリがインストールされたので、モデルに適した Explainer を取得しましょう。Explainer には多くの種類があります。この例では、適切な [SHAP](https://github.com/slundberg/shap) モデル Explainer を呼び出すことによって、多くの種類のモデルを説明するために使用できる「ブラック ボックス」の説明である表形式の *Explainer* を使用します。

In [ ]:
from interpret.ext.blackbox import TabularExplainer

# 「フィーチャー」と「クラス」フィールドはオプションです
tab_explainer = TabularExplainer(model, 
                             X_train, 
                             features=features, 
                             classes=labels)
print(tab_explainer, "ready!")

### グローバル フィーチャーの重要度を取得する

最初に行うことは、全体*フィーチャーの重要度*を評価することによってモデルを説明しようとすることです - つまり、各フィーチャーがトレーニング データセット全体に基づいて予測に影響を与える程度を定量化します。

In [ ]:
# ここでトレーニング データまたはテスト データを使用できます
global_tab_explanation = tab_explainer.explain_global(X_train)

# 重要度別の上位フィーチャーを取得する
global_tab_feature_importance = global_tab_explanation.get_feature_importance_dict()
for feature, importance in global_tab_feature_importance.items():
    print(feature,":", importance)

フィーチャーの重要度が順位付けされ、最も重要な機能が最初に表示されます。

### ローカル フィーチャーの重要度を取得する

全体的な見解がありますが、個々の観察を説明はどうですか? 可能性のある各ラベル値を予測する決定に各機能が影響を与えた程度を定量化して、個々の予測の*ローカル*説明を生成しましょう。この場合、バイナリ モデルであるため、2 つのラベル (糖尿病以外と糖尿病) があります。また、データセット内の個々の観測値に対するこれらのラベル値の各フィーチャーの影響を定量化できます。テスト データセットの最初の 2 つのケースを評価するだけです。

In [ ]:
# 説明したい観測値を取得する (最初の 2 つ)
X_explain = X_test[0:2]

# 予測を取得する
predictions = model.predict(X_explain)

# ローカルな説明を取得する
local_tab_explanation = tab_explainer.explain_local(X_explain)

# 各ラベルのフィーチャー名と重要度を取得する
local_tab_features = local_tab_explanation.get_ranked_local_names()
local_tab_importance = local_tab_explanation.get_ranked_local_values()

for l in range(len(local_tab_features)):
    print('Support for', labels[l])
    label = local_tab_features[l]
    for o in range(len(label)):
        print("\tObservation", o + 1)
        feature_list = label[o]
        total_support = 0
        for f in range(len(feature_list)):
            print("\t\t", feature_list[f], ':', local_tab_importance[l][o][f])
            total_support += local_tab_importance[l][o][f]
        print("\t\t ----------\n\t\t Total:", total_support, "Prediction:", labels[predictions[o]])



## Azure ML モデル トレーニング実験に説明可能性を追加する

これまで見てきたように、Azure ML の外部でトレーニングされたモデルの説明を生成できます。ただし、Azure ML ワークスペースでモデルをトレーニングするために実験を使用する場合は、モデルの説明を生成してログに記録できます。

まず、Azure ML SDK を使用してワークスペースに接続する必要があります。

> **注**: 前回の演習を完了してから Azure サブスクリプションとの認証済みセッションの有効期限が切れている場合は、再認証を求めるメッセージが表示されます。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 保存した構成ファイルからワークスペースを読み込む
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

### 実験を使用してモデルをトレーニングおよび説明する

では、実験を作成して、必要なファイルをローカル フォルダーに配置しましょう - この場合、糖尿病データの同じ CSV ファイルを使用してモデルをトレーニングします。

In [ ]:
import os, shutil
from azureml.core import Experiment

# 実験ファイル用フォルダーを作成する
experiment_folder = 'diabetes_train_and_explain'
os.makedirs(experiment_folder, exist_ok=True)

# データ ファイルを実験フォルダーにコピーする
shutil.copy('data/diabetes.csv', os.path.join(experiment_folder, "diabetes.csv"))

次のフィーチャーを含む以外 Azure ML トレーニング スクリプトと同様のトレーニング スクリプトを作成します。

- 以前使用したモデルの説明を生成する同じライブラリがインポートされ、グローバルな説明を生成するために使用されます
- **ExplanationClient** ライブラリを使用して、説明を実験出力にアップロードします

In [ ]:
%%writefile $experiment_folder/diabetes_training.py
# ライブラリをインポートする
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# Azure ML 実行ライブラリをインポートする
from azureml.core.run import Run

# モデルの説明用ライブラリをインポートする
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient
from interpret.ext.blackbox import TabularExplainer

# 実験実行コンテキストを取得する
run = Run.get_context()

# 糖尿病データセットを読み込む
print("Loading Data...")
data = pd.read_csv('diabetes.csv')

features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
labels = ['not-diabetic', 'diabetic']

# フィーチャーとラベルを分離する
X, y = data[features].values, data['Diabetic'].values

# データをトレーニング セットとテスト セットに分割する
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# デシジョン ツリー モデルをトレーニングする
print('Training a decision tree model')
model = DecisionTreeClassifier().fit(X_train, y_train)

# 精度を計算する
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
run.log('Accuracy', np.float(acc))

# AUC を計算する
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
# 出力フォルダーに保存されたファイルは、自動的に実験レコードにアップロードされます
joblib.dump(value=model, filename='outputs/diabetes.pkl')

# 説明を取得する
explainer = TabularExplainer(model, X_train, features=features, classes=labels)
explanation = explainer.explain_global(X_test)

# 説明クライアントを取得し、説明をアップロードする
explain_client = ExplanationClient.from_run(run)
explain_client.upload_model_explanation(explanation, comment='Tabular Explanation')

# 実行を完了する
run.complete()

これで、Estimator を使用してトレーニング スクリプトを実行し、実験を実行できます。スクリプトが **TabularExplainer** を作成できるように **azureml-interpret** ライブラリがトレーニング環境に含まれており、スクリプトが **ExplainerClient** クラスを使用できるように **azureml-contrib-interpret** パッケージが含まれていることに注意してください。

In [ ]:
from azureml.train.estimator import Estimator
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails

# 実験用 Python 環境を作成する
env = Environment('diabetes-interpret-env')
env.python.user_managed_dependencies = False # 依存関係を Azure ML に管理させる
env.docker.enabled = True # ドッカー コンテナーを使用する

# パッケージの依存関係のセットを作成する (azureml-contrib-interpret パッケージを含む)
packages = CondaDependencies.create(conda_packages=['scikit-learn','pandas'],
                                    pip_packages=['azureml-defaults','azureml-interpret','azureml-contrib-interpret'])

# 環境に依存関係を追加する
env.python.conda_dependencies = packages

# Estimator を作成する
estimator = Estimator(source_directory=experiment_folder,
              compute_target = 'local', # ローカル コンピューティングを使用する
              environment_definition = env,
              entry_script='diabetes_training.py')

# 実験を実行する
experiment = Experiment(workspace = ws, name = 'diabetes_train_and_explain')
run = experiment.submit(config=estimator)
RunDetails(run).show()
run.wait_for_completion()

## フィーチャーの重要度値を取得する

実験の実行が完了したら、**ExplanationClient** クラスを使用して、実行用に登録された説明からフィーチャーの重要度を取得できます。

In [ ]:
from azureml.contrib.interpret.explanation.explanation_client import ExplanationClient

# フィーチャーの説明を取得する
client = ExplanationClient.from_run(run)
engineered_explanations = client.download_model_explanation()
feature_importances = engineered_explanations.get_feature_importance_dict()

# 全体フィーチャーの重要度
print('Feature\tImportance')
for key, value in feature_importances.items():
    print(key, '\t', value)

## Azure Machine Learning Studio でモデルの説明を表示する

また、実行の詳細ウィジェットのリンクをクリックして、Azure Machine Learning studio で実行を確認し、**説明**タブを表示することもできます。そして:

1.**表形式の説明** Explainerを選択します。
2.全体的なグローバル フィーチャーの重要度を示す**グローバル重要度**グラフを表示します。
3.**サマリー重要度**グラフを表示すると、*スウォーム*、*バイオリン*、*ボックス*プロットのテスト データの各データ ポイントが表示されます。
4.個々のポイントを選択すると、選択したデータ ポイントの個々の予測の**ローカル フィーチャーの重要度** が表示されます。


**詳細情報**: Azure ML での Explainer の使用の詳細については、[ドキュメント](https://docs.microsoft.com/azure/machine-learning/how-to-machine-learning-interpretability)を参照してください。 